In [ ]:
#エッジ用のnumpy保存プログラム 2023年度用
import numpy as np
from PIL import Image
import cv2
import os,glob,random

outfile="./2023npz/2023num40karnel3-384px.npz"#変更必要
max_photo=40000
photo_size=384#変更必要
x=[]
y=[]
laplace3=[[1,1,1],
         [1,-8,1],
         [1,1,1]]

def main():
    glob_files("../2023hands/aizawa/right/omote",0)
    glob_files("../2023hands/aizawa/right/ura",1)
    glob_files("../2023hands/arai/right/omote",2)
    glob_files("../2023hands/arai/right/ura",3)
    glob_files("../2023hands/ebato/right/omote",4)
    glob_files("../2023hands/ebato/right/ura",5)
    glob_files("../2023hands/hasegawa/right/omote",6)
    glob_files("../2023hands/hasegawa/right/ura",7)
    glob_files("../2023hands/higuchi/right/omote",8)
    glob_files("../2023hands/higuchi/right/ura",9)
    glob_files("../2023hands/isii/right/omote",10)
    glob_files("../2023hands/isii/right/ura",11)
    glob_files("../2023hands/kojima/right/omote",12)
    glob_files("../2023hands/kojima/right/ura",13)
    glob_files("../2023hands/nakamura/right/omote",14)
    glob_files("../2023hands/nakamura/right/ura",15)
    glob_files("../2023hands/oono/right/omote",16)
    glob_files("../2023hands/oono/right/ura",17)
    glob_files("../2023hands/sakai/right/omote",18)
    glob_files("../2023hands/sakai/right/ura",19)
    glob_files("../2023hands/sekine/right/omote",20)
    glob_files("../2023hands/sekine/right/ura",21)
    glob_files("../2023hands/shimada/right/omote",22)
    glob_files("../2023hands/shimada/right/ura",23)
    glob_files("../2023hands/shimizu/right/omote",24)
    glob_files("../2023hands/shimizu/right/ura",25)
    glob_files("../2023hands/takahashi/right/omote",26)
    glob_files("../2023hands/takahashi/right/ura",27)
    glob_files("../2023hands/tanakahiroki/right/omote",28)
    glob_files("../2023hands/tanakahiroki/right/ura",29)
    glob_files("../2023hands/tanakamiyu/right/omote",30)
    glob_files("../2023hands/tanakamiyu/right/ura",31)
    glob_files("../2023hands/tanakataiki/right/omote",32)
    glob_files("../2023hands/tanakataiki/right/ura",33)
    glob_files("../2023hands/tokuyama/right/omote",34)
    glob_files("../2023hands/tokuyama/right/ura",35)
    glob_files("../2023hands/watanabe/right/omote",36)
    glob_files("../2023hands/watanabe/right/ura",37)
    glob_files("../2023hands/yamaguchi/right/omote",38)
    glob_files("../2023hands/yamaguchi/right/ura",39)
    np.savez(outfile,x=x,y=y)
    print("保存しました:"+outfile,len(x))
    print(x[0],y[0])
    
def glob_files(path,label):
    files=glob.glob(path+"/*.jpg")
    random.shuffle(files)
    num=0
    for f in files:
        if num>=max_photo: break
        num+=1
        
        #---------edgesの時のプログラム--------------
        img=cv2.imread(f)
        img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        karnel=np.array(laplace3 , np.float64)
        edges=cv2.filter2D(img_gray,-1,karnel)
        edges=cv2.resize(edges,(photo_size,photo_size))
        
        
        edges=np.asarray(edges)#1つの配列に格納（この場合2つ）
        x.append(edges)
        y.append(label)
if __name__=='__main__':
    main()

In [ ]:
%%time
#エッジ検出を用いた機械学習
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.summary()
    return model

def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


im_rows = 384 # 変更必要
im_cols = 384 # 変更必要
im_color = 1
in_shape = (im_rows, im_cols, im_color)
nb_classes = 40 #変更必要

photos = np.load('./2023npz/2023num40karnel3-384px.npz')#変更必要
x = photos['x']
y = photos['y']

x = x.reshape(-1, im_rows, im_cols, im_color)
x = x.astype('float32') / 255
y = keras.utils.to_categorical(y.astype('int32'), nb_classes)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.9)

model = get_model(in_shape, nb_classes)


hist = model.fit(x_train, y_train,
          batch_size=256,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])


plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model.save_weights('./2023hdf/2023num40karnel3-384pxbatch256.hdf5') #変更必要

In [ ]:
%%time
#エッジ検出を用いた機械学習
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.summary()
    return model

def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


im_rows = 256 # 変更必要
im_cols = 256 # 変更必要
im_color = 1
in_shape = (im_rows, im_cols, im_color)
nb_classes = 40 #変更必要

photos = np.load('./2023npz/2023num40karnel3-256px.npz')#変更必要
x = photos['x']
y = photos['y']

x = x.reshape(-1, im_rows, im_cols, im_color)
x = x.astype('float32') / 255
y = keras.utils.to_categorical(y.astype('int32'), nb_classes)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.9)

model = get_model(in_shape, nb_classes)


hist = model.fit(x_train, y_train,
          batch_size=256,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])


plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model.save_weights('./2023hdf/2023num40karnel3-256pxbatch256.hdf5') #変更必要

In [ ]:
%%time
#エッジ検出を用いた機械学習
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.summary()
    return model

def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


im_rows = 64 # 変更必要
im_cols = 64 # 変更必要
im_color = 1
in_shape = (im_rows, im_cols, im_color)
nb_classes = 40 #変更必要

photos = np.load('./2023npz/2023num40karnel3-64px.npz')#変更必要
x = photos['x']
y = photos['y']

x = x.reshape(-1, im_rows, im_cols, im_color)
x = x.astype('float32') / 255
y = keras.utils.to_categorical(y.astype('int32'), nb_classes)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.9)

model = get_model(in_shape, nb_classes)


hist = model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])


plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model.save_weights('./2023hdf/2023num40karnel3-64pxbatch128.hdf5') #変更必要

In [ ]:
%%time
#エッジ検出を用いた機械学習
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.summary()
    return model

def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


im_rows = 64 # 変更必要
im_cols = 64 # 変更必要
im_color = 1
in_shape = (im_rows, im_cols, im_color)
nb_classes = 40 #変更必要

photos = np.load('./2023npz/2023num40karnel3-64px.npz')#変更必要
x = photos['x']
y = photos['y']

x = x.reshape(-1, im_rows, im_cols, im_color)
x = x.astype('float32') / 255
y = keras.utils.to_categorical(y.astype('int32'), nb_classes)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.9)

model = get_model(in_shape, nb_classes)


hist = model.fit(x_train, y_train,
          batch_size=256,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])


plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model.save_weights('./2023hdf/2023num40karnel3-64pxbatch256.hdf5') #変更必要

In [ ]:
#他人受入率の評価プログラム
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image


def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model


def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model

#他人受入率用のテスト配列
omote=["gotou/omote",
        "isii/omote",
        "mimura/omote",
       "nakamura/omote",
        "okutani/omote",
        "saitouhide/omote",
        "saitouryou/omote",
        "sakuma/omote",
        "tanaka/omote",
        "tochikubo/omote"]
ura=["gotou/ura",
     "isii/ura",
     "mimura/ura",
      "nakamura/ura",
      "okutani/ura",
      "saitouhide/ura",
      "saitouryou/ura",
      "sakuma/ura",
      "tanaka/ura",
      "tochikubo/ura"]

target_omote=[]
target_ura=[]
laplace3=[[1,1,1],
         [1,-8,1],
         [1,1,1]]
passpercent=[]
notpasspercent=[]
omotepercent=[]
urapercent=[]


#12-25修正
for k in range(10):#変更必要
    for i in range(10):
        target_omote.append("../graytest/"+str(omote[k])+"/"+str(i)+".jpg")#変更必要
        target_ura.append("../graytest/"+str(ura[k])+"/"+str(i)+".jpg")#変更必要


im_rows = 16 #変更必要
im_cols = 16 #変更必要
im_color = 1 #変更必要
in_shape = (im_rows, im_cols, im_color)
nb_classes = 40 #変更必要
#配列の最大値　11月追加
max_score = 0.0

#12-25修正
#ラベル
LABELS=["相澤さんの表","相澤さんの裏",
       "新井さんの表","新井さんの裏",
       "江波戸の表","江波戸の裏",
       "長谷川さんの表","長谷川さんの裏",
       "樋口さんの表","樋口さんの裏",
       "石井さんの表","石井さんの裏",
       "小島さんの表","小島さんの裏",
       "中村さんの表","中村さんの裏",
       "大野さんの表","大野さんの裏",
       "坂井さんの表","坂井さんの裏",
       "関根さんの表","関根さんの裏",
       "島田さんの表","島田さんの裏",
       "清水さんの表","清水さんの裏",
       "高橋さんの表","高橋さんの裏",
       "田中ひろきさんの表","田中ひろきさんの裏",
       "田中みゆさんの表","田中みゆさんの裏",
       "田中たいきさんの表","田中たいきさんの裏",
       "徳山さんの表","徳山さんの裏",
       "渡邉さんの表","渡邉さんの裏",
       "山口さんの表","山口さんの裏"]

model = get_model(in_shape, nb_classes)
model.load_weights('./2023hdf/2023num40karnel3-16pxbatch256.hdf5')#変更必要

#最大値を見つける関数 11月追加
def findmaxscore(per):
    global max_score
    if per!=100.0:
        if max_score<=per:
            max_score=per 

def check_photo(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    karnel=np.array(laplace3 , np.float64)#変更必要
    img=cv2.filter2D(img,-1,karnel)
    img=cv2.resize(img,(im_rows,im_cols))
    plt.imshow(img,cmap="gray")
    plt.show()

    x=np.asarray(img)
    x=x.reshape(-1,im_rows,im_cols,im_color)
    x=x/255
    
    #予測
    pre=model.predict([x])[0]#画像の予測
    idx=pre.argmax()
    per=(pre[idx]*100)
    #----------97.9---98.5---98.9---99.5---99.9---99,95---99.99-------------
    if per < 99.99 and ("omote" in path): #閾値変更箇所
        omotepercent.append(per)
    elif per < 99.99 and ("ura" in path): #閾値変更箇所
        urapercent.append(per)
    else:
        pass
    return(idx,per)

def check_photo_str(path):
    idx,per=check_photo(path)
    if per >= 99.99: #閾値変更箇所
        print("この写真は、",LABELS[idx],"です。")
        print("可能性は、",per,"%です。")
        passpercent.append(per)
        #配列の最大値を見つける　11月追加
        findmaxscore(per)
    else:
        print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
        print("認証の結果は",LABELS[idx],"でした。")
        notpasspercent.append(per)
    
if __name__=='__main__':
    number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
    for i in range(100):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_omote[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    for i in range(100):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_ura[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
                
    print(passpercent)
    print(notpasspercent)
    print("他人受入に用いる個数"+str(len(passpercent)))
    FAR=len(passpercent)
    print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
    print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
    print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
    totalpass=len(passpercent)/200 #12-26修正
    totalnotpass=len(notpasspercent)/200
    omotenotpass=len(omotepercent)/100
    omotepass=(100-len(omotepercent))/100
    uranotpass=len(urapercent)/100
    urapass=(100-len(urapercent))/100
    print("================================================")
    #未学習の手を閾値未満で識別した割合
    print("他人拒否率"+str(round(totalnotpass*100))+"%")
    #未学習の手を閾値以上で識別した割合
    print("他人受入率"+str(round(totalpass*100))+"%")
    print("表の手だけで未学習の手を閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
    print("表の手だけで未学習の手を閾値以上で識別した割合"+str(round(omotepass*100))+"%")
    print("裏の手だけで未学習の手を閾値未満で識別した割合"+str(round(uranotpass*100))+"%")
    print("裏の手だけで未学習の手を閾値以上で識別した割合"+str(round(urapass*100))+"%")
    print("閾値以上の最大値は"+str(max_score)) #11月追加
    try:
        print("閾値以上の最小値は"+str(min(passpercent))) #11月追加 
    except:
        pass

In [ ]:
#本人拒否率の評価プログラム
# import keras 
# from keras.models import Sequential
# from keras.layers import Dense,Dropout,Flatten
# from keras.layers import Conv2D,MaxPooling2D
# from keras.optimizers import RMSprop
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# import numpy as np
# import cv2
# from PIL import Image


# def def_model(in_shape, nb_classes):
#     model = Sequential()
#     model.add(Conv2D(32,
#               kernel_size=(3, 3),
#               activation='relu',
#               input_shape=in_shape))
#     model.add(Conv2D(32, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))

#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))

#     model.add(Flatten())
#     model.add(Dense(512, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(nb_classes, activation='softmax'))
#     return model


# def get_model(in_shape, nb_classes):
#     model = def_model(in_shape, nb_classes)
#     model.compile(
#         loss='categorical_crossentropy',
#         optimizer=RMSprop(),
#         metrics=['accuracy'])
#     return model


# #本人受入率用のテスト配列
# omote=["aizawa/right/omote",
#       "arai/right/omote",
#       "ebato/right/omote",
#       "hasegawa/right/omote",
#       "higuchi/right/omote",
#       "isii/right/omote",
#       "kojima/right/omote",
#       "nakamura/right/omote",
#       "oono/right/omote",
#       "sakai/right/omote",
#       "sekine/right/omote",
#       "shimada/right/omote",
#       "shimizu/right/omote",
#       "takahashi/right/omote",
#       "tanakahiroki/right/omote",
#       "tanakamiyu/right/omote",
#       "tanakataiki/right/omote",
#       "tokuyama/right/omote",
#       "watanabe/right/omote",
#       "yamaguchi/right/omote"]
# ura=["aizawa/right/ura",
#       "arai/right/ura",
#       "ebato/right/ura",
#       "hasegawa/right/ura",
#       "higuchi/right/ura",
#       "isii/right/ura",
#       "kojima/right/ura",
#       "nakamura/right/ura",
#       "oono/right/ura",
#       "sakai/right/ura",
#       "sekine/right/ura",
#       "shimada/right/ura",
#       "shimizu/right/ura",
#       "takahashi/right/ura",
#       "tanakahiroki/right/ura",
#       "tanakamiyu/right/ura",
#       "tanakataiki/right/ura",
#       "tokuyama/right/ura",
#       "watanabe/right/ura",
#       "yamaguchi/right/ura"]
# target_omote=[]
# target_ura=[]
# laplace3=[[1,1,1],
#          [1,-8,1],
#          [1,1,1]]
# passpercent=[]
# notpasspercent=[]
# omotepercent=[]
# urapercent=[]
# notIncludeLABEL=[]

# for k in range(20):#変更必要
#     for i in range(10):
#         target_omote.append("../2023graytest/"+str(omote[k])+"/gray"+str(i)+".jpg")#変更必要
#         target_ura.append("../2023graytest/"+str(ura[k])+"/gray"+str(i)+".jpg")#変更必要


# im_rows = 32 #変更必要
# im_cols = 32 #変更必要
# im_color = 1 #変更必要
# in_shape = (im_rows, im_cols, im_color)
# nb_classes = 40 #変更必要
# #配列の最大値
# max_score = 0.0

# #ラベル
# LABELS=["相澤さんの表","相澤さんの裏",
#        "新井さんの表","新井さんの裏",
#        "江波戸の表","江波戸の裏",
#        "長谷川さんの表","長谷川さんの裏",
#        "樋口さんの表","樋口さんの裏",
#        "石井さんの表","石井さんの裏",
#        "小島さんの表","小島さんの裏",
#        "中村さんの表","中村さんの裏",
#        "大野さんの表","大野さんの裏",
#        "坂井さんの表","坂井さんの裏",
#        "関根さんの表","関根さんの裏",
#        "島田さんの表","島田さんの裏",
#        "清水さんの表","清水さんの裏",
#        "高橋さんの表","高橋さんの裏",
#        "田中ひろきさんの表","田中ひろきさんの裏",
#        "田中みゆさんの表","田中みゆさんの裏",
#        "田中たいきさんの表","田中たいきさんの裏",
#        "徳山さんの表","徳山さんの裏",
#        "渡邉さんの表","渡邉さんの裏",
#        "山口さんの表","山口さんの裏"]
# AbsLABELS=["相澤さん","新井さん","江波戸","長谷川さん","樋口さん",
#            "石井さん","小島さん","中村さん","大野さん","坂井さん",
#            "関根さん","島田さん","清水さん","高橋さん","田中ひろきさん",
#            "田中みゆさん","田中たいきさん","徳山さん","渡邉さん","山口さん"]

# model = get_model(in_shape, nb_classes)
# model.load_weights('./2023hdf/2023num40karnel3-32pxbatch256.hdf5')#変更必要

# def findmaxscore(per):
#     global max_score
#     if per!=100.0:
#         if max_score<=per:
#             max_score=per

# def check_photo(path):
#     img=cv2.imread(path)
#     img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     karnel=np.array(laplace3 , np.float64)#変更必要
#     img=cv2.filter2D(img,-1,karnel)
#     img=cv2.resize(img,(im_rows,im_cols))
#     plt.imshow(img,cmap="gray")
#     plt.show()

#     x=np.asarray(img)
#     x=x.reshape(-1,im_rows,im_cols,im_color)
#     x=x/255
    
#     #予測
#     pre=model.predict([x])[0]#画像の予測
#     idx=pre.argmax()
#     per=(pre[idx]*100)
#     #----------97.9---98.5---98.9---99.5---99.9---99,95---99.99-------------
#     if per<99.99999 and ("omote" in path): #閾値変更箇所
#         omotepercent.append(per)
#     elif per<99.99999 and ("ura" in path): #閾値変更箇所
#         urapercent.append(per)
#     else:
#         pass       
#     return(idx,per)

# def check_photo_str(path,i):
#     idx,per=check_photo(path)
#     if per>=99.99999: #閾値変更箇所
#         print("この写真は、",LABELS[idx],"です。")
#         #閾値以上なのに識別を間違えているときの処理
#         if i < 10:
#             ConfLABEL = AbsLABELS[0]
#         elif i < 20:
#             ConfLABEL = AbsLABELS[1]
#         elif i < 30:
#             ConfLABEL = AbsLABELS[2]
#         elif i < 40:
#             ConfLABEL = AbsLABELS[3]
#         elif i < 50:
#             ConfLABEL = AbsLABELS[4]
#         elif i < 60:
#             ConfLABEL = AbsLABELS[5]
#         elif i < 70:
#             ConfLABEL = AbsLABELS[6]
#         elif i < 80:
#             ConfLABEL = AbsLABELS[7]
#         elif i < 90:
#             ConfLABEL = AbsLABELS[8]
#         elif i < 100:
#             ConfLABEL = AbsLABELS[9]
#         elif i < 110:
#             ConfLABEL = AbsLABELS[10]
#         elif i < 120:
#             ConfLABEL = AbsLABELS[11]
#         elif i < 130:
#             ConfLABEL = AbsLABELS[12]
#         elif i < 140:
#             ConfLABEL = AbsLABELS[13]
#         elif i < 150:
#             ConfLABEL = AbsLABELS[14]
#         elif i < 160:
#             ConfLABEL = AbsLABELS[15]
#         elif i < 170:
#             ConfLABEL = AbsLABELS[16]
#         elif i < 180:
#             ConfLABEL = AbsLABELS[17]
#         elif i < 190:
#             ConfLABEL = AbsLABELS[18]
#         elif i < 200:
#             ConfLABEL = AbsLABELS[19]
  
#         if "表" in LABELS[idx]:
#             if ConfLABEL not in LABELS[idx] :
#                 notIncludeLABEL.append("-" + ConfLABEL + "を" + LABELS[idx] + "だと誤認識-")  
#                 print('識別エラーが起きています'+'\033[31m')
#         if "裏" in LABELS[idx]:
#             if ConfLABEL not in LABELS[idx]:
#                 notIncludeLABEL.append("-" + ConfLABEL + "を" + LABELS[idx] + "だと誤認識-")  
#                 print('識別エラーが起きています'+'\033[31m')        
#         #-------------------------------------------
#         print("可能性は、",per,"%です。")
#         passpercent.append(per)
#         #配列の最大値を見つける
#         findmaxscore(per)
#     else:
#         print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
#         print("認証の結果は",LABELS[idx],"でした。")
#         notpasspercent.append(per)
    
# if __name__=='__main__':
#     number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
#     for i in range(200):
#         print("\nここからは"+str(i)+"番目の写真です")
#         check_photo_str(target_omote[i],i)
#         for l in range(15):
#             if i==number[l]:
#                 print("================================================")
#                 print("テスト対象の人が変わります")
#                 print("================================================")
#     for i in range(200):
#         print("\nここからは"+str(i)+"番目の写真です")
#         check_photo_str(target_ura[i],i)
#         for l in range(15):
#             if i==number[l]:
#                 print("================================================")
#                 print("テスト対象の人が変わります")
#                 print("================================================")

#     print(passpercent)
#     print(notpasspercent)
#     print("閾値以上で識別したテスト数"+str(len(passpercent)))
#     print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
#     print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
#     print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
#     totalpass=len(passpercent)/400
#     totalnotpass=len(notpasspercent)/400
#     omotenotpass=len(omotepercent)/200
#     omotepass=(200-len(omotepercent))/200
#     uranotpass=len(urapercent)/200
#     urapass=(200-len(urapercent))/200
#     print("================================================")
#     #閾値未満で識別した割合
#     print("本人拒否率"+str(round(totalnotpass*100))+"%")
#     #閾値以上で識別した割合
#     print("本人受入率"+str(round(totalpass*100))+"%")
#     print("表の手だけを閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
#     print("表の手だけを閾値以上で識別した割合"+str(round(omotepass*100))+"%")
#     print("裏の手だけを閾値未満で識別した割合"+str(round(uranotpass*100))+"%")
#     print("裏の手だけを閾値以上で識別した割合"+str(round(urapass*100))+"%")
#     print("閾値以上の最大値は"+str(max_score))
#     try:
#         print("閾値以上の最小値は"+str(min(passpercent))) #11月追加
#     except:
#         pass
#     #他人受入率の評価と本人拒否率の評価すべてにおける他人受入
#     print(notIncludeLABEL)
#     print("全体の他人受入率は" + str(round((FAR+len(notIncludeLABEL))/600*100)) + "%")
#     print(FAR)
#     print(len(notIncludeLABEL))                